# MCP Python Code Execution with Docker on Clarifai Local Runner

This notebook demonstrates how to integrate an MCP server with Docker-based Python code execution capabilities running on a Clarifai Local Runner with an LLM.

## What is MCP and How Does it Work with LLMs?

MCP (Model Context Protocol) servers provide tools that LLMs can call to perform specific tasks. This notebook shows the complete integration workflow:

1. **Deploy MCP Server on Local Runner**: Your MCP server hosts tools (like Python code execution in Docker containers) on your local machine using Clarifai Local Runner
2. **Deploy LLM**: An LLM (like GPT) is deployed and accessible via API on Clarifai
3. **Connect the Two**: You write client code that:
   - Sends user queries to the LLM along with available MCP tools
   - The LLM decides if it needs to use a tool to answer the query
   - Your code executes the tool call via the MCP server running locally
   - Results are sent back to the LLM for a final natural language response

## Local Runner vs Cloud Deployment

This Docker-based version runs on your local machine using Clarifai Local Runner, which:
- **Runs locally**: MCP server runs on your machine with Docker
- **Uses Docker**: Provides isolated Python code execution in containers
- **Better security**: Code runs in isolated containers on your infrastructure
- **Local resources**: Uses your local machine's compute and Docker daemon

## Prerequisites

- Clarifai account with PAT (Personal Access Token)
- Docker installed and running on your local machine
- Clarifai Local Runner set up
- MCP server with Docker-based code execution deployed via Local Runner
- Python 3.11+

## Step 1: Deploy MCP Server with Local Runner

This MCP server requires deployment using Clarifai Local Runner on your local machine.

### Setup Steps:

1. **Install Docker**: Ensure Docker is installed and running on your local machine
2. **Set up Local Runner**: Follow the Clarifai Local Runner documentation: https://docs.clarifai.com/compute/local-runners/
3. **Deploy MCP Server**: Use the command:
   ```bash
   clarifai model upload local-runner
   ```
   This deploys the MCP server to run on your local machine via Local Runner

4. **Verify Deployment**: Ensure the MCP server is active and accessible locally

The Local Runner approach allows the MCP server to access your local Docker daemon for isolated code execution.

## Step 2: Install Dependencies

Install required packages for MCP client and OpenAI integration.

In [ ]:
!pip install clarifai==11.7.5 fastmcp pydantic openai

## Step 3: Configuration

Set up your Clarifai PAT and model endpoints.

Note: The MCP server runs locally via Local Runner, but you still connect to it through Clarifai's MCP API endpoint.

In [ ]:
import os

# TODO: Set your Clarifai PAT
os.environ["CLARIFAI_PAT"] = "your_pat_here"  # Replace with your actual PAT

# LLM model endpoint (GPT-OSS on Clarifai)
MODEL_NAME = "https://clarifai.com/openai/chat-completion/models/gpt-oss-120b"

# TODO: Update with your deployed MCP server information
USER_ID = "your_user_id"      # Replace with your Clarifai user ID
APP_ID = "your_app_id"        # Replace with your MCP app ID
MODEL_ID = "your_model_id"    # Replace with your MCP model ID

MCP_SERVER_URL = f"https://api.clarifai.com/v2/ext/mcp/v1/users/{USER_ID}/apps/{APP_ID}/models/{MODEL_ID}"

## Step 4: Import Required Libraries

Import all necessary modules for MCP client, OpenAI client, and async operations.

In [ ]:
import asyncio
import json
from typing import Any, Dict

from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport
from openai import AsyncOpenAI

# Initialize OpenAI client pointing to Clarifai
openai_client = AsyncOpenAI(
    api_key=os.environ["CLARIFAI_PAT"], 
    base_url="https://api.clarifai.com/v2/ext/openai/v1"
)

# Global variables for MCP client
mcp_client = None
mcp_tools = []

## Step 5: Connect to MCP Server

This function establishes a connection to your MCP server running on Local Runner.

The connection goes through Clarifai's API, but the MCP server itself runs locally with access to your Docker daemon.

In [ ]:
async def connect_to_mcp():
    """Connect to the Python code execution MCP server."""
    global mcp_client, mcp_tools

    transport = StreamableHttpTransport(
        url=MCP_SERVER_URL,
        headers={"Authorization": "Bearer " + os.environ["CLARIFAI_PAT"]}
    )

    mcp_client = Client(transport)
    await mcp_client.__aenter__()

    # Get available tools from MCP server
    tools_result = await mcp_client.list_tools()
    mcp_tools = [
        {
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.inputSchema,
            },
        }
        for tool in tools_result
    ]

    return mcp_tools

## Step 6: Execute MCP Tool Calls

This function handles calling MCP tools when the LLM decides to use them.

In [ ]:
async def execute_mcp_tool(tool_name: str, arguments: Dict[str, Any]) -> str:
    """Execute an MCP tool call."""
    try:
        result = await mcp_client.call_tool(tool_name, arguments)
        return result.content[0].text
    except Exception as e:
        return f"Error executing {tool_name}: {str(e)}"

## Step 7: Test with Docker-Based Code Execution Queries

This function demonstrates the full workflow:
1. Send query to LLM with available MCP tools
2. LLM decides if it needs to use a tool and generates code
3. Parse LLM response and execute tool calls via MCP (in Docker containers)
4. Send tool results back to LLM for final answer

In [ ]:
async def test_queries():
    """
    Test with code execution queries.
    
    This function demonstrates the full LLM + MCP integration workflow:
    1. Send a user query to the LLM with available MCP tools in context
    2. The LLM analyzes the query and decides whether to use a tool
    3. If a tool is needed, extract the tool call details (including generated code) and execute via MCP in Docker
    4. Send tool results back to the LLM to generate a natural language response
    """

    test_queries = [
        "Use time package in python to print the current time",
        "Use the numpy package to perform a matrix multiplication and print the result"
    ]

    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*50}")
        print(f"Query {i}: {query}")
        print('='*50)

        try:
            # Step 1: Send query to LLM with MCP tools available
            # The LLM receives both the user question AND the list of available tools
            # It can now decide: "Do I need to write and execute code for this?"
            response = await openai_client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {
                        "role": "system",
                        "content": "You are a Python assistant with code execution tools."
                    },
                    {"role": "user", "content": query}
                ],
                tools=mcp_tools,  # This tells the LLM what tools are available
                tool_choice="auto",  # Let the LLM decide if it needs to use a tool
                temperature=0.1,
                max_tokens=500
            )

            message = response.choices[0].message
            print(f"Assistant: {message.content}")

            # Step 2: Check if LLM wants to use tools
            # The LLM's response includes tool_calls if it decided a tool is needed
            if message.tool_calls:
                print(f"\nTool calls: {len(message.tool_calls)}")

                # Step 3: Execute each tool call via MCP
                # The LLM has generated Python code and specified which tool to use
                # Now we execute those tool calls using our MCP client (code runs in Docker)
                tool_responses = []
                for tool_call in message.tool_calls:
                    print(f"\n--- {tool_call.function.name} ---")
                    
                    # Extract the arguments the LLM wants to pass to the tool
                    args = json.loads(tool_call.function.arguments)

                    # Display what the LLM generated
                    if "code" in args:
                        print(f"Code: {args['code']}")
                    if "packages" in args:
                        print(f"Packages: {args['packages']}")

                    # Call the MCP server to execute the Python code in a Docker container
                    result = await execute_mcp_tool(tool_call.function.name, args)
                    tool_responses.append(result)
                    print(f"Result:\n{result}")

                # Step 4: Send tool results back to LLM for final response
                # Now the LLM has the code execution results and can generate a natural answer
                follow_up_messages = [
                    {"role": "system", "content": "You are a Python assistant with code execution tools."},
                    {"role": "user", "content": query},
                    message  # Include the LLM's original response with tool calls
                ]

                # Add the tool results to the conversation
                for tool_call, tool_result in zip(message.tool_calls, tool_responses):
                    follow_up_messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": tool_result
                    })

                # Send everything back to the LLM to generate a final natural language answer
                final_response = await openai_client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=follow_up_messages,
                    temperature=0.1,
                    max_tokens=300
                )

                print(f"\nFinal: {final_response.choices[0].message.content}")

        except Exception as e:
            print(f"Error: {str(e)}")

## Step 8: Cleanup Function

Clean up the MCP connection when done.

In [ ]:
async def cleanup():
    """Clean up MCP connection."""
    global mcp_client
    if mcp_client:
        await mcp_client.__aexit__(None, None, None)
        mcp_client = None

## Step 9: Run the Demo

Execute the complete workflow:
1. Connect to the deployed MCP server
2. Test with code execution queries
3. Cleanup

In [ ]:
async def main():
    """Main function"""
    print("Testing GPT-OSS with Docker-Based Python Execution Tools")

    try:
        # Connect to MCP server
        await connect_to_mcp()
        print(f"Connected! Tools: {len(mcp_tools)}")

        # Test the tools
        await test_queries()

    finally:
        await cleanup()

# Run the async main function
await main()

## Summary

This notebook demonstrates the three-part integration with Docker-based execution on Local Runner:

1. **MCP Connection**: Establish connection to MCP server deployed on Local Runner and retrieve available tools
2. **LLM Query**: Send test queries to LLM with MCP tools in context so LLM understands what is available and can decide whether to call tools
3. **Tool Execution**: Parse LLM response, execute tool calls via MCP client running on Local Runner with Docker, and return results to LLM

## Local Runner Benefits:

- **Local Execution**: MCP server runs on your machine via Local Runner
- **Docker Access**: Direct access to your local Docker daemon for container execution
- **Isolation**: Each code execution runs in a separate Docker container
- **Security**: Code runs in isolated containers on your infrastructure
- **Package Management**: Easy installation of Python packages per execution
- **Control**: Full control over compute resources and execution environment

The workflow enables LLMs to dynamically execute Python code through your locally-hosted MCP server in secure Docker environments.